In [1]:
# Import the libraries
import pygama.lgdo.lh5_store as lh5 
from legendmeta import LegendMetadata
from datetime   import datetime, timezone #for datetime functions
import pandas as pd # pandas
import numpy as np # numpy
import time

starttime = time.time()

In [2]:
# Specify the path of the file. The following path will work for NERSC.
tcm_file = "/global/cfs/cdirs/m2676/data/lngs/l200/public/prodenv/prod-blind/ref/v02.00/generated/tier/tcm/cal/p03/r000/l200-p03-r000-cal-20230312T033419Z-tier_tcm.lh5"

In [3]:
print("List of tcm file elements:")
print(lh5.ls(tcm_file))
print("--------------------------")
print("The keys it contains:")
lh5.ls(tcm_file,'hardware_tcm_1/')

List of tcm file elements:
['hardware_tcm_1']
--------------------------
The keys it contains:


['hardware_tcm_1/array_id',
 'hardware_tcm_1/array_idx',
 'hardware_tcm_1/cumulative_length']

In [4]:
# Since the keys have different sizes, we will load up the array_id and array_idx into one dataframe ...
hitindex_df = lh5.load_dfs(tcm_file, ['array_id', 'array_idx'], 'hardware_tcm_1/')
hitindex_df

,array_id,array_idx
0,1116805.0,0
1,1116805.0,1
2,1118403.0,0
3,1116805.0,2
4,1118404.0,0
...,...,...
983796,1089603.0,14613
983797,1115204.0,42650
983798,1084803.0,20227
983799,1105602.0,9159


In [5]:
# ...and the cumulative length into another dataframe
cumulative_length_df = lh5.load_dfs(tcm_file, ['cumulative_length'], 'hardware_tcm_1/')
# Make a cumulative length array
for (columnName, columnData) in cumulative_length_df.items():
    cl_array = columnData.values

In [6]:
# Making the channel map
lmeta = LegendMetadata()
# Channel map
dt    = datetime(2023, 4, 22, 5, 54, 22, tzinfo=timezone.utc)
chmap = lmeta.hardware.configuration.channelmaps.on(dt)
# Detectors dictionary
detlist=[]
for det, val in chmap.items():
    detlist.append(det)
channel_dict = {}
i=0
for channel_name, channel_data in chmap.items():
    try:
        channel_dict[channel_data['daq']['rawid']] = (channel_name, channel_data.system)
        i=i+1
    except:
        #print(channel_name)
        continue
print(f"The entries in the dictionary:{len(detlist)}.")

The entries in the dictionary:234.


In [7]:
# Convert the detectors dictionary into a pandas dataframe
channel_df = pd.DataFrame.from_dict(channel_dict, orient='index', columns=['channel_name', 'system'])
channel_df

,channel_name,system
1027200,BSLN01,bsln
1027201,PULS01,puls
1027202,MUON01,auxs
1027203,PULS01ANA,puls
1104000,V02160A,geds
...,...,...
2004818,PMT607,pmts
2004819,PMT608,pmts
2004820,PMT609,pmts
2004821,PMT610,pmts


In [8]:
# Merge the dataframes into a master dataframe
master_hitindex  = pd.merge(hitindex_df, channel_df, left_on='array_id', right_index=True, how='left')
master_hitindex

,array_id,array_idx,channel_name,system
0,1116805.0,0,V09724A,geds
1,1116805.0,1,V09724A,geds
2,1118403.0,0,V02166B,geds
3,1116805.0,2,V09724A,geds
4,1118404.0,0,V04199A,geds
...,...,...,...,...
983796,1089603.0,14613,P00748A,geds
983797,1115204.0,42650,V01415A,geds
983798,1084803.0,20227,V00048A,geds
983799,1105602.0,9159,V07647A,geds


In [9]:
# Define a coincidence dataframe
coin_df = pd.DataFrame(columns=['event_no', 'channel_name','raw_id','hit_id','system'])

In [10]:
start = 0
stop = 0
evt = 0
xprev = 1
for x in cl_array:
    stop = x - xprev + start
    eventno = evt
    temp = master_hitindex.iloc[start:stop]
    channel_name = np.array(temp["channel_name"])
    rawid = np.array(temp["array_id"])
    hitid = np.array(temp["array_idx"])
    system = np.array(temp["system"])
    evt = evt + 1
    xprev=x
    start = stop
    coin_df.loc[evt] = [eventno, channel_name, rawid, hitid, system]
# set index using column
coin_df = coin_df.set_index('event_no')

KeyboardInterrupt: 

In [ ]:
coin_df

In [ ]:
endtime = time.time()

In [ ]:
print(f"Time taken: {endtime - starttime} s.")